In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()

iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)

iris_frame['target'] = iris.target

# Adding the SimplePipeline from the previous notebook

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Load the dataset and perform train test split."""
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.train()

# Adding functionality to our previous Pipeline

In [3]:
from sklearn.preprocessing import StandardScaler


class PipelineWithDataEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        
        self.scaler = StandardScaler()
        
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.apply_scaler()
        self.train()

In [4]:
pipeline = PipelineWithDataEngineering()
pipeline.run_pipeline()
accuracy_score = pipeline.get_accuracy()
print(f'current model accuracy is: {accuracy_score}')

current model accuracy is: 0.9591836734693877


# Unit Testing

In [5]:
import unittest


class TestIrisDataEngineering(unittest.TestCase):
    def setUp(self):
        self.pipeline = PipelineWithDataEngineering()
        self.pipeline.load_dataset()
    
    def test_scaler_preprocessing_brings_x_train_mean_near_zero(self):
        original_mean = self.pipeline.X_train.stack().mean()
        
        self.pipeline.apply_scaler()
        
        self.assertTrue(original_mean > self.pipeline.X_train.mean())
        self.assertAlmostEqual(self.pipeline.X_train.mean(), 0.0, places=3)
        print(f'Original X train mean: {original_mean}')
        print(f'Transformed X train mean: {self.pipeline.X_train.mean()}')
        
    def test_scaler_preprocessing_brings_x_train_std_near_one(self):
        self.pipeline.apply_scaler()
        
        self.assertAlmostEqual(self.pipeline.X_train.std(), 1.0, places=3)
        print(f'Transformed X train standard deviation : {self.pipeline.X_train.std()}')

In [6]:
import sys

suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisDataEngineering)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.035s

OK


Original X train mean: 3.5889423076923075
Transformed X train mean: -5.978123978750843e-17
Transformed X train standard deviation : 1.0


<unittest.runner.TextTestResult run=2 errors=0 failures=0>